In [1]:
import requests
from bs4 import BeautifulSoup 

In [2]:
url = 'https://www.theguardian.com/world/2017/aug/07/anti-atheist-prejudice-secularity'
text = requests.get(url).text

The one below is a github code for downloading comments. Do not use this; instead use a modified version of this in the cell below this one.

In [4]:
import urllib2
import re
import time
import sys

# This script will download all comments from a Guardian (theguardian.com) article into a single .html file
# Use 'File ID' to easily identify your file later (this could also be something like a Citavi identifier)
# Ingo Kleiber <ikleiber@googlemail.com>
# https://gist.github.com/IngoKl/3ebd968ef3c6da406638

print """\
   _____                     _ _             _____  _      
  / ____|                   | (_)           |  __ \| |     
 | |  __ _   _  __ _ _ __ __| |_  __ _ _ __ | |  | | |     
 | | |_ | | | |/ _` | '__/ _` | |/ _` | '_ \| |  | | |     
 | |__| | |_| | (_| | | | (_| | | (_| | | | | |__| | |____ 
  \_____|\__,_|\__,_|_|  \__,_|_|\__,_|_| |_|_____/|______|
"""                                                         

print 'Guardian Comment Downloader v.1.1 - Ingo Kleiber (05.01.2015)\n'
guardianLink = raw_input ('> Guadian Link: ')
fileId = raw_input('> File ID: ')

#Figuring out the PageId
pageResponse = urllib2.urlopen(guardianLink)
commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
# print "Comments Match = ", commentsMatch()
if commentsMatch:
    pageId = commentsMatch.group(1)
    print '[+] pageId has been retrieved ('+pageId+')'
else:
    sys.exit('[-] Could not retrieve pageId!')

#Retrieving comments
downloadCount = 1
downloadError = 0

startTime = time.time()
fileName = 'GuardianDownload_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
f = open(fileName, 'a')


print '[~] Start downloading pages ...'
while downloadError == 0:
    try:
        response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
        html = response.read()
        f.write(html)
        if downloadCount == 1:
            soup = BeautifulSoup(html, 'html.parser')
        print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
        downloadCount = downloadCount + 1
        
    except:
        downloadError = 1

stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
f.close() 

   _____                     _ _             _____  _      
  / ____|                   | (_)           |  __ \| |     
 | |  __ _   _  __ _ _ __ __| |_  __ _ _ __ | |  | | |     
 | | |_ | | | |/ _` | '__/ _` | |/ _` | '_ \| |  | | |     
 | |__| | |_| | (_| | | | (_| | | (_| | | | | |__| | |____ 
  \_____|\__,_|\__,_|_|  \__,_|_|\__,_|_| |_|_____/|______|

Guardian Comment Downloader v.1.1 - Ingo Kleiber (05.01.2015)

> Guadian Link: https://www.theguardian.com/commentisfree/2017/aug/10/will-even-white-people-die-how-to-explain-nuclear-war-to-your-kids
> File ID: test
[+] pageId has been retrieved (72634)
[~] Start downloading pages ...
[+] Page: 1 has been downloaded!
[+] Page: 2 has been downloaded!
[+] Page: 3 has been downloaded!
[+] Page: 4 has been downloaded!
[+] Page: 5 has been downloaded!
[+] Page: 6 has been downloaded!
[+] Page: 7 has been downloaded!
[+] Page: 8 has been downloaded!
[+] Page: 9 has been downloaded!
[+] Page: 10 has been downloaded!
[+] Page: 11 has been 

In [5]:
# soup = BeautifulSoup(html, 'html.parser')

In [6]:
# print soup.prettify()

In [13]:
soup.comment_body

In [25]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})
replies = soup.findAll("span", {"class" : "d-comment__reply-to-author"})
for user, comment, recommend, reply in zip(users, comments, recommends, replies):
    print user.text, comment.text, recommend.text, reply.text


Reality_Man 
Again let's stick with what people understand what is the percentage of CO2 in the atmosphere? 0.04%. CO2 is not a problem. Enviormental hacks are making a living from this. All government funded.
0 JulesBywaterLees
WilliamHBonney 
Refundable deposits...And only one thousand dollars.
Tesla is living off government grants- given who lives in the White House right now, that may not continue.
0 ID9499946
ID9499946 
You don't just sound like a moron..
Why don't you turn your ire on oil production? How many lives has that cost? How many people die prematurely? I suppose you are in favour of Trump reopening coal mines too?
0 mothercourage
ID9499946 
Yeah but pre-orders of 500,000 for the new Tesla blows that theory out of the water - try again!
0 WilliamHBonney
JulesBywaterLees 
cobalt mining is a by product of copper and nickle mining- so if you object to cobalt mining then steer clear of all electrical gadgets as they contain a lot of copper.
0 mothercourage
JulesBywaterLees 


a class="social__action social-icon-wrapper" data-link-name="social-comment : facebook" href="https://www.facebook.com/dialog/share?app_id=180444840287&amp;href=https%3A%2F%2Fdiscussion.theguardian.com%2Fcomment-permalink%2F103303986&amp;quote=alexandrerizvin%20commented%3A%20%22My%20fantasy%20of%20freedom%20and%20independence%20has%20been%20to%20have%20no%20dependence%20on%20cars.%22" target="_blank">

In [1]:
# import requests
url = "https://www.theguardian.com/us/commentisfree"

list_of_urls = requests.get(url).text

In [2]:
# from bs4 import BeautifulSoup 
soup2 = BeautifulSoup(list_of_urls, "html.parser")

In [4]:
# soup2.prettify()

In [17]:
import re
i = 0
for link in soup2.findAll('a', attrs={'href': re.compile("https://www.theguardian.com/commentisfree/2017")}):
    print link.get('href')
    i += 1
print i

https://www.theguardian.com/commentisfree/2017/aug/10/finally-democrats-are-looking-in-the-mirror-thats-reason-for-optimism
https://www.theguardian.com/commentisfree/2017/aug/10/finally-democrats-are-looking-in-the-mirror-thats-reason-for-optimism
https://www.theguardian.com/commentisfree/2017/aug/10/minnesota-mosque-bombing-donald-trump-silence
https://www.theguardian.com/commentisfree/2017/aug/10/minnesota-mosque-bombing-donald-trump-silence
https://www.theguardian.com/commentisfree/2017/aug/10/west-gripped-venezuela-ignore-brazil-jeremy-corbyn-nicolas-maduro
https://www.theguardian.com/commentisfree/2017/aug/10/west-gripped-venezuela-ignore-brazil-jeremy-corbyn-nicolas-maduro
https://www.theguardian.com/commentisfree/2017/aug/09/pregnancy-can-kill-right-to-chose-jessica-valenti
https://www.theguardian.com/commentisfree/2017/aug/09/pregnancy-can-kill-right-to-chose-jessica-valenti
https://www.theguardian.com/commentisfree/2017/aug/09/god-save-us-from-donald-trumps-fire-and-fury
https